In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torchvision
from torchvision import models
import os
import time
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader 
import torch.optim as optim
import torchvision.transforms as tt
from torch.utils.data import random_split
from torchvision.utils import make_grid

import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# preparing and check dataset
dataset_dir = '../input/covid-19-xray-image-dataset-with-huge-samples/COVID/'
train_dir = dataset_dir + '/train'
test_dir = dataset_dir + '/test'

classes = os.listdir(train_dir)
print(classes)
# transform to tensor and std, mean calcutation
# train and val data transform
# training process

In [3]:
def check_numbers(imgs_path):
    for cls in os.listdir(imgs_path):
        print(f"Class {cls} has len {len(os.listdir(imgs_path + '/' + cls))}")
        
folders = [train_dir, test_dir]

for fol in folders:
    print(fol.split('/')[-1])
    check_numbers(fol)

In [4]:
# https://discuss.pytorch.org/t/computing-the-mean-and-std-of-dataset/34949

# def CalcMeanAndSTD(dataset):
#     loader = DataLoader(dataset, batch_size = 10, num_workers = 0, shuffle = False)

#     mean = 0.
#     std = 0.
#     for images, _ in loader:
#         # Rearrange batch to be the shape of [B, C, W * H]
#         batch_samples = images.size(0)
#         images = images.view(batch_samples, images.size(1), -1)       
#         # Compute mean and std here
#         mean += images.mean(2).sum(0)
#         std += images.std(2).sum(0)

#     mean /= len(loader.dataset)
#     std /= len(loader.dataset)
    
#     return mean, std

# mean_vals, std_vals = CalcMeanAndSTD(dataset)
# print(mean_vals, std_vals)

In [5]:
def data_transform(data_type = None):
    if data_type == train_dir:
        data_T = tt.Compose([
            tt.Resize(size = (200,200)),
            tt.RandomHorizontalFlip(),
            tt.RandomGrayscale(),
            tt.ToTensor()
#             tt.Normalize(mean=mean_vals, std = std_vals)
        ])
    
    elif data_type == test_dir:
        data_T = tt.Compose([
            tt.Resize(size = (200,200)),
            tt.ToTensor()
#             tt.Normalize(mean=mean_vals, std = std_vals)
        ])
        
    return data_T

train_data = ImageFolder(train_dir, transform = data_transform(train_dir))
test_data = ImageFolder(test_dir, transform = data_transform(test_dir))

In [6]:
print(len(train_data))
img, label = train_data[0]
print(img.shape, label)
img

In [7]:
def show_image(img, label):
    print(f'Dataset {train_data.classes[label]} - {str(label)}')
    plt.imshow(img.permute(1,2,0))

# img,label = dataset[0]
# show_image(img,label)
show_image(*train_data[200])

In [8]:
for i in range(5): 
    image,label = train_data[i]
    print(image.shape,label)

In [9]:
val_size = 400
train_size = len(train_data) - val_size

train_dataset, val_dataset = random_split(train_data, [train_size, val_size])
len(train_dataset), len(val_dataset)

In [10]:
train_batch = DataLoader(train_dataset, batch_size = 32, shuffle = True, num_workers = 2, pin_memory = True)
valid_batch = DataLoader(val_dataset, batch_size= 64, shuffle = True, num_workers=2, pin_memory=True)
test_batch = DataLoader(test_data, batch_size = 32, shuffle = False)

In [11]:
def show_batch(batch):
    for images, labels in batch:
        fig, ax = plt.subplots(figsize = (20,25))
        ax.set_xticks([])
        ax.set_yticks([])
        ax.imshow(make_grid(images, nrow = 16).permute(1, 2, 0))
        break

show_batch(train_batch)

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [13]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
    
def fit(epochs, model, train_loader, val_loader, criterion, optimizer, scheduler):
    torch.cuda.empty_cache()
    
    train_losses = []
    test_losses = []
    train_scores = []
    val_score = []
    lrs = []
    
    fit_time = time.time()
    for epoch in range(epochs):
        since = time.time()
        running_loss = 0
        train_score = 0
        
        for image,label in train_loader:
            
            model.train()
            image = image.to(device)
            label = label.to(device)
            output = model(image)
            
            # accuracy calculation
            ps = torch.exp(output)
            _, top_class = ps.topk(1, dim = 1)
            correct = top_class == label.view(*top_class.shape)
            train_score += torch.mean(correct.type(torch.FloatTensor))
            
            loss = criterion(output, label)
            
            loss.backward()
            
            optimizer.step()
            optimizer.zero_grad()
            
            scheduler.step()
            lrs.append(get_lr(optimizer))
            running_loss += loss.item()
            
        else:
            model.eval()
            test_loss = 0
            scores = 0
            #validation loop#
            with torch.no_grad():
                for image, label in val_loader:
                    image = image.to(device); label = label.to(device);

                    output = model(image)

                    #accuracy calulcation
                    ps = torch.exp(output)
                    _, top_class = ps.topk(1, dim=1)
                    correct = top_class == label.view(*top_class.shape)
                    scores += torch.mean(correct.type(torch.FloatTensor))
                    #loss
                    loss = criterion(output, label)                                  
                    test_loss += loss.item()
            
            #calculation mean for each batch
            train_losses.append(running_loss/len(train_loader))
            test_losses.append(test_loss/len(val_loader))
            train_scores.append(train_score/len(train_loader))
            val_score.append(scores/len(val_loader))

            print("Epoch: {}/{}.. ".format(epoch+1, epochs),
                  "Train Loss: {:.3f}.. ".format(running_loss/len(train_loader)),
                  "Val Loss: {:.3f}.. ".format(test_loss/len(val_loader)),
                  "Train acc Score: {:.3f}.. ".format(train_score/len(train_loader)),
                  "Val acc : {:.3f}.. ".format(scores/len(val_loader)),
                  "Lr: {:.4f} ".format(get_lr(optimizer)),
                  "Time: {:.2f}s" .format(time.time()-since)
                 )
        
    history = {'train_loss' : train_losses, 'val_loss': test_losses, 
               'train_acc': train_scores, 'val_acc':val_score, 'lrs': lrs}
    print('Total time: {:.2f} m' .format((time.time()- fit_time)/60))
    return history

# Mobilenet_v2 model

In [14]:
output_label = 2

model_mobile = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)

model_mobile.classifier = nn.Sequential(nn.Linear(in_features=1280, out_features=output_label))

model_mobile.to(device)
# model_mobile

In [15]:
max_lr = 0.0001
epoch = 20
weight_decay = 1e-4

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_mobile.parameters(), lr=max_lr, weight_decay=weight_decay)
sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epoch, 
                                            steps_per_epoch=len(train_batch))

history_mobile = fit(epoch, model_mobile, train_batch, valid_batch, criterion, optimizer, sched)

In [16]:
def plot_loss(history, n_epoch):
    epoch = [x for x in range(1, n_epoch+1)]
    plt.plot(epoch, history['train_loss'], label='Train_loss')
    plt.plot(epoch, history['val_loss'], label='val_loss')
    plt.title('Loss per epoch')
    plt.ylabel('Loss')
    plt.xlabel('epoch')
    plt.legend() 
    plt.show()

def plot_score(history, n_epoch):
    epoch = [x for x in range(1, n_epoch+1)]
    plt.plot(epoch, history['train_acc'], label='Train_acc')
    plt.plot(epoch, history['val_acc'], label='val_acc')
    plt.title('Accuracy per epoch')
    plt.ylabel('score')
    plt.xlabel('epoch')
    plt.legend()
    plt.show()

def plot_lr(history):
    plt.plot(history['lrs'], label='learning rate')
    plt.title('One Cycle Learning Rate')
    plt.ylabel('Learning Rate')
    plt.xlabel('steps')
    plt.legend() 
    plt.show()

In [17]:
plot_score(history_mobile, epoch)
plot_loss(history_mobile, epoch)
plot_lr(history_mobile)

# Resnet18 Model

In [18]:
resnet_model = models.resnet18(pretrained = True)
num_fits = resnet_model.fc.in_features

resnet_model.fc = nn.Linear(num_fits, 2)
resnet_model = resnet_model.to(device)


max_lr = 0.0001
epoch = 20
weight_decay = 1e-4

criterion = nn.CrossEntropyLoss()
# Observe that all parameters are being optimized
optimizer = optim.Adam(resnet_model.parameters(), lr = max_lr, 
                       weight_decay=weight_decay)

sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=max_lr, 
                                            epochs = epoch, steps_per_epoch=len(train_batch))

history_resnet18 = fit(epoch, resnet_model, train_batch, valid_batch, criterion, optimizer, sched)

In [19]:
plot_score(history_resnet18, epoch)
plot_loss(history_resnet18, epoch)
plot_lr(history_resnet18)

# Model Evaluation

In [34]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

target_labels = {
    0: 'Covid Positive', 
    1: 'Covid Negative'
}

def predict_dataset(dataset, model):
    model.eval()
    model.to(device)
    torch.cuda.empty_cache()
    predict = []
    y_true = []
    
    for image, label in dataset:
        image = image.unsqueeze(0)
        image = image.to(device)
 
        output = model(image)
        ps = torch.exp(output)
        _, top_class = ps.topk(1, dim = 1)
        
        predic = np.squeeze(top_class.cpu().numpy())
        predict.append(predic)
        y_true.append(label)
        
    return list(y_true), list(np.array(predict).reshape(1, -1).squeeze(0))


def report(y_true, y_predict, title:str):
    print(classification_report(y_true, y_predict))
    sns.heatmap(confusion_matrix(y_true, y_predict), annot = True)
    plt.xticks(ticks = np.arange(0.5, len(target_labels)), labels = list(target_labels.values()), 
               rotation = 45)
    
    plt.yticks(ticks = np.arange(0.5, len(target_labels)), 
               labels = list(target_labels.values()), rotation = 0)
    
    plt.title(title)
    

In [32]:
y_true, y_predict = predict_dataset(test_data, model_mobile)
report(y_true, y_predict, title='Mobilenet_v2 Test Set')


In [38]:
y_true, y_predict = predict_dataset(test_data, resnet_model)
report(y_true, y_predict, title = 'VGG16 Test Set')